# Real Time 2D Finite Volume Code - Explicit Method

In [ ]:
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GL import *
from matplotlib import cm
import numpy as np
import numexpr as ne
import networkx as nx


name = 'Explicit 2D FVM'

def get_cmap_from_matplotlib(cmap=cm.Reds):
    ncol = cmap.N
    cmap_rgba = []
    for i in xrange(ncol):
        b, g, r, _ = cmap(i) # Not sure why this is inverted, I was expecting r, g, b.
        cmap_rgba.append(int(255.0) << 24 | 
            (int(float(r) * 255.0) << 16) | 
            (int(float(g) * 255.0) << 8 ) | 
            (int(float(b) * 255.0) << 0 ) )

    return np.array(cmap_rgba), len(cmap_rgba)


###### Flow definition #########################################################
Lx, Ly = 1.0, 1.0
ni, nj = 200, 200

dx, dy = Lx / ni, Ly / nj

Ax = dy * 1.0
Ay = dx * 1.0
A = Ax

k = 100.0
rho = 1000.0
cp = 4200.0
dV = dx * dy * 1.0
q = 1000.0 / dV

# Condition for positive coefficients 
# dt < rho * c * dx^2 / 2k (for 2D)
dt = 0.9 * rho * cp * (dx ** 2) / (4.0 * k)

def to_continuous(ij):
    i,j = ij
    return i + j * ni
    
zero_I = np.zeros(nj)
zero_J = np.zeros(ni)
last_I = (ni-1) * np.ones(nj)
last_J = (nj-1) * np.ones(ni)
I = np.arange(0,ni)
J = np.arange(0,nj)

left  =  np.array([zero_I,J]).astype(int).T
right =  np.array([last_I,J]).astype(int).T
north =  np.array([I,last_J]).astype(int).T
south =  np.array([I,zero_J]).astype(int).T

graph = nx.grid_2d_graph(ni,nj)

subgraph_left  = graph.subgraph(map(tuple,  left))
subgraph_right = graph.subgraph(map(tuple, right))
subgraph_north = graph.subgraph(map(tuple, north))
subgraph_south = graph.subgraph(map(tuple, south))

T_left  = 10.0
T_right = 10.0
T_north = 10.0
T_south = 10.0

initial_temperature = 50.0
T     = initial_temperature * np.ones((ni, nj))
T_old = np.copy(T)

edges = np.array([e for e in graph.edges_iter()])

e0 = edges[:,0]
e1 = edges[:,1]
e0 = [e0[:,0],e0[:,1]]
e1 = [e1[:,0],e1[:,1]]

nodes = np.array([n for n in graph.nodes_iter()])
nodes = [nodes[:,0], nodes[:,1]]

nodes_left  = np.array([n for n in subgraph_left.nodes_iter() ])
nodes_left = [nodes_left[:,0], nodes_left[:,1]]

nodes_right = np.array([n for n in subgraph_right.nodes_iter()])
nodes_right = [nodes_right[:,0], nodes_right[:,1]]

nodes_north = np.array([n for n in subgraph_north.nodes_iter()])
nodes_north = [nodes_north[:,0], nodes_north[:,1]]

nodes_south = np.array([n for n in subgraph_south.nodes_iter()])
nodes_south = [nodes_south[:,0], nodes_south[:,1]]

###### OpenGL Properties #######################################################
solid = np.ones((nj, ni))
plot_rgba = np.zeros(ni*nj, dtype=np.uint32)
cmap_rgba, ncol = get_cmap_from_matplotlib(cmap=cm.jet)

iter_per_frame = 1

def solve_fvm(sources=[]):
    for i in xrange(iter_per_frame):
        T_old[:] = T[:]  

        T[nodes] = rho * cp * dV * T_old[nodes] / dt

        #T_old_n = T_old[nodes]
        #T[nodes] = ne.evaluate("rho * cp * dV * T_old_n / dt")

        # apply boundary conditions
        T[nodes_left ] += -2.0 * k * A / dx * (T_old[nodes_left ] - T_left )
        T[nodes_right] += -2.0 * k * A / dx * (T_old[nodes_right] - T_right)
        T[nodes_north] += -2.0 * k * A / dx * (T_old[nodes_north] - T_north)
        T[nodes_south] += -2.0 * k * A / dx * (T_old[nodes_south] - T_south)        

        #T_old_e0 = T_old[e0]
        #T_old_e1 = T_old[e1]
        #flux = ne.evaluate("k * A / dx * (T_old_e0 - T_old_e1)")

        flux = k * A / dx * (T_old[e0] - T_old[e1])  
        np.add.at(T, e0, -flux)
        np.add.at(T, e1,  flux)

        T[sources] += q * dV 

        T[nodes] *= dt / (rho * cp * dV)

    return (T.T).flatten()        

    
def display():
    '''
    Set upper and lower limits for plotting.
    Do one FVM step.
    
    '''   
    sources = (1 - solid.T).astype(np.bool)
    plotvar = solve_fvm(sources=sources)
        
    # convert the plotvar array into an array of colors to plot
    # if the mesh point is solid, make it black
    minvar = 9.0
    maxvar = 1.1 * np.max(plotvar)

    frac = (plotvar[:] - minvar)/(maxvar - minvar)

    icol = frac * ncol    
    plot_rgba[:] = solid.flatten().astype(np.int) * cmap_rgba[icol.astype(np.int)] 
    
    # Fill the pixel buffer with the plot_rgba array
    glBufferData(GL_PIXEL_UNPACK_BUFFER, plot_rgba.nbytes, plot_rgba, GL_STREAM_COPY)

    # Copy the pixel buffer to the texture, ready to display
    glTexSubImage2D(GL_TEXTURE_2D,0,0,0,ni,nj,GL_RGBA,GL_UNSIGNED_BYTE, None)
    
    # Render one quad to the screen and colour it using our texture
    # i.e. plot our plotvar data to the screen
    glClear(GL_COLOR_BUFFER_BIT)
    glBegin(GL_QUADS)
    
    x0, y0 = 0.0, 0.0
    x1, y1 = ni, nj
    
    glTexCoord2f(0.0, 0.0)
    glVertex3f(x0, y0, 0.0)
    
    glTexCoord2f(1.0, 0.0)
    glVertex3f(x1, y0, 0.0)
    
    glTexCoord2f (1.0, 1.0)
    glVertex3f(x1, y1, 0.0)
    
    glTexCoord2f (0.0, 1.0)
    glVertex3f(x0, y1, 0.0)
    
    glEnd()
    glutSwapBuffers()     


def resize(w,h):
    '''
    GLUT resize callback to allow us to change the window size.
    
    '''
    global width, height
    width = w
    height = h
    glViewport (0, 0, w, h)
    glMatrixMode (GL_PROJECTION)
    glLoadIdentity()
    glOrtho(0., ni, 0., nj, -200. ,200.)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    
def mouse(button, state, x, y):    
    global draw_solid_flag, ipos_old, jpos_old
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:       
        draw_solid_flag = 0
        xx = x
        yy = y
        ipos_old = int(float(xx) / width * float(ni))
        jpos_old = int(float(height - yy) / height * float(nj))
        
    if button == GLUT_RIGHT_BUTTON and state == GLUT_DOWN:        
        draw_solid_flag = 1
        xx = x
        yy = y
        ipos_old = int(float(xx) / width * float(ni))
        jpos_old = int(float(height - yy) / height * float(nj))


def mouse_motion(x,y):
    '''
    GLUT call back for when the mouse is moving
    This sets the solid array to draw_solid_flag as set in the mouse callback
    It will draw a staircase line if we move more than one pixel since the
    last callback - that makes the coding a bit cumbersome:
    '''
    global ipos_old, jpos_old
    
    xx = x
    yy = y
    ipos = int(float(xx) / width * float(ni))
    jpos = int(float(height-yy) / height * float(nj))

    if ipos <= ipos_old:
        i1 = ipos
        i2 = ipos_old
        j1 = jpos
        j2 = jpos_old
    
    else:
        i1 = ipos_old
        i2 = ipos
        j1 = jpos_old
        j2 = jpos
    
    jlast = j1   
    
    for i in xrange(i1, i2+1):
        if i1 != i2:
            frac = (i-i1) / (i2-i1)
            jnext = (frac * (j2-j1)) + j1        
        else:
            jnext=j2
        
        if jnext >= jlast:
            solid[jlast,i] = draw_solid_flag
            
            for j in xrange(jlast, jnext+1):    
                solid[j,i] = draw_solid_flag
        else:
            solid[jlast,i] = draw_solid_flag
            for j in xrange(jnext, jlast+1):
                solid[j,i] = draw_solid_flag
        
        jlast = jnext
        
    ipos_old=ipos
    jpos_old=jpos


glutInit(name)
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
glutInitWindowSize(ni, nj)
glutInitWindowPosition(50, 50)
glutCreateWindow(name)

glClearColor(1.0,1.0,1.0,1.0)
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
glOrtho(0,ni,0.,nj, -200.0, 200.0)

glEnable(GL_TEXTURE_2D)

gl_Tex = glGenTextures(1)
glBindTexture(GL_TEXTURE_2D, gl_Tex)

glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA8, ni, nj, 0, GL_RGBA, GL_UNSIGNED_BYTE, None)

gl_PBO = glGenBuffers(1)
glBindBuffer(GL_PIXEL_UNPACK_BUFFER, gl_PBO)

#setup callbacks
glutDisplayFunc(display)
glutReshapeFunc(resize)
glutIdleFunc(display)
glutMouseFunc(mouse)
glutMotionFunc(mouse_motion)

glutMainLoop()